In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/support-documentation/support_documentation.pdf


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict


In [3]:
!pip install -qU langchain-core langchain-text-splitters langchain 
!pip install -qU transformers sentence-transformers chromadb pypdf langchain
!pip install -qU "langchain[cohere]"
!pip install -qU langchain-community
%config Completer.use_jedi = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 73.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [4]:
!pip install -qU chromadb
!pip install -qU "langchain-chroma>=0.1.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 14.2 MB/s eta 0:00:0000:01


In [5]:
from langchain_chroma import Chroma

In [6]:
from kaggle_secrets import UserSecretsClient

CO_API_KEY = UserSecretsClient().get_secret("COHERE_API_KEY")

In [7]:
from langchain.chat_models import init_chat_model
model = init_chat_model("command-r-plus", 
                        model_provider="cohere",
                        cohere_api_key=CO_API_KEY)

In [8]:
response = model.invoke("Tell me a short story about a curious cat")

In [9]:
print(response.content)

Once upon a time, there was a curious cat named Whiskers. He lived in a cozy house with his owner, Mrs. Smith, who adored him dearly. Whiskers spent his days exploring the nooks and crannies of their home, always on the lookout for something new and exciting.

One sunny morning, Whiskers was wandering around the garden when he noticed a small hole under the fence. Intrigued, he poked his head through and found himself in a mysterious new world. It was the neighbouring garden, full of intriguing smells and sounds.

Whiskers decided to embark on a little adventure. He carefully squeezed his fluffy body through the hole and emerged on the other side, his green eyes sparkling with curiosity. He began to explore his new surroundings, sniffing at the flowers, chasing butterflies, and climbing trees.

As he wandered further, he stumbled upon a small pond glittering in the sunlight. Fascinated by the shimmering water, Whiskers crept closer, his paws padding softly on the grass. Suddenly, he sp

In [10]:
!pip uninstall -y ypy-websocket
!pip install langchain-core "langgraph>0.2.27" 
!pip install -Uq "unstructured-client<0.30.0"
!pip uninstall -y aiofiles
!pip install "aiofiles>=24.1.0"
!pip install langchain-community
!pip install pypdf
!pip install "unstructured[pdf]"
!pip install chromadb

Found existing installation: ypy-websocket 0.8.4
Uninstalling ypy-websocket-0.8.4:
  Successfully uninstalled ypy-websocket-0.8.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sigstore 3.6.1 requires rich~=13.0, but you have rich 14

In [11]:
!pip install chromadb

# Define loader, text splitters 
The next step is to get the loader and the text spliters so that we can load
and split the text document to get into a array of Documents

In [14]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


In [15]:
loader = DirectoryLoader(path="..", glob="**/*.pdf", 
                         recursive=True, 
                         show_progress=True)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
docs: Document = loader.load_and_split()

100%|██████████| 1/1 [00:11<00:00, 11.41s/it]


In [18]:
from langchain_cohere.embeddings import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=CO_API_KEY)

In [22]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name = "default_collection",
    embedding_function = embeddings,
    persist_directory="./default_collection_db",
)

In [27]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]

In [31]:
doc_ids = vector_store.add_documents(uuids = uuids, documents = docs)
doc_ids

['33de532b-7f07-46db-8733-c868a752f96b',
 '26204d6a-389a-4540-be83-d4f141df5248']